In [2]:
# -- Built-in modules -- #
import os
import gc
import sys

# -- Third-part modules -- #
import json
import matplotlib.pyplot as plt
import numpy as np
import torch
import xarray as xr
from tqdm.notebook import tqdm
import psutil
import tracemalloc
import pickle 

# --Proprietary modules -- #
from functions import chart_cbar, r2_metric, f1_metric, compute_metrics
from loaders import AI4ArcticChallengeTestDataset
from unet import UNet
from utils import CHARTS, SIC_LOOKUP, SOD_LOOKUP, FLOE_LOOKUP, SCENE_VARIABLES, colour_str
from unet import FeatureMap
# %store -r train_options
train_options = pickle.load(open('models/unet_attention/version_512_0/train_options.pkl', 'rb'))


In [3]:
# Get GPU resources.
if torch.cuda.is_available():
    print(colour_str('GPU available!', 'green'))
    print('Total number of available devices: ', colour_str(torch.cuda.device_count(), 'orange'))
    device = torch.device(f"cuda:{train_options['gpu_id']}")

else:
    print(colour_str('GPU not available.', 'red'))
    device = torch.device('cpu')

GPU available!
Total number of available devices:  1


In [4]:
print('Loading model.')
# Setup U-Net model, adam optimizer, loss function and dataloader.
# net = UNet(options=train_options).to(device)
from unet_attention import UNetAttention
net = UNetAttention(options=train_options).to(device)

model_name = 'best_model.pt'
model_path = f'models/unet_attention/version_512_0/{model_name}'

net.load_state_dict(torch.load(model_path)['model_state_dict'])

print('Model successfully loaded.')

Loading model.
Model successfully loaded.


In [12]:
train_options['val_patch_size'] = None
with open(train_options['path_to_env'] + 'datalists/testset.json') as file:
    train_options['test_list'] = json.loads(file.read())
train_options['test_list'] = [file[17:32] + '_' + file[77:80] + '_prep.nc' for file in train_options['test_list']]
train_options['path_to_processed_data'] += ''  # The test data is stored in a separate folder inside the training data.
upload_package = xr.Dataset()  # To store model outputs.
dataset = AI4ArcticChallengeTestDataset(options=train_options, files=train_options['test_list'], test=True)
asid_loader = torch.utils.data.DataLoader(dataset, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)
print('Setup ready')

Setup ready


In [13]:
train_options['path_to_processed_data']

'data'

In [14]:
print('Testing.')
os.makedirs('inference', exist_ok=True)
net.eval()
for inf_x, _, masks, scene_name in tqdm(iterable=asid_loader, total=len(train_options['test_list']), colour='green', position=0):
    scene_name = scene_name[:19]  # Removes the _prep.nc from the name.
    torch.cuda.empty_cache()
    inf_x = inf_x.to(device, non_blocking=True)

    with torch.no_grad(), torch.cuda.amp.autocast():
        output = net(inf_x)

    for chart in train_options['charts']:
        output[chart] = torch.argmax(output[chart], dim=1).squeeze().cpu().numpy()
        upload_package[f"{scene_name}_{chart}"] = xr.DataArray(name=f"{scene_name}_{chart}", data=output[chart].astype('uint8'),
                                                               dims=(f"{scene_name}_{chart}_dim0", f"{scene_name}_{chart}_dim1"))

    # - Show the scene inference.
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(10, 10))
    for idx, chart in enumerate(train_options['charts']):
        ax = axs[idx]
        output[chart] = output[chart].astype(float)
        output[chart][masks] = np.nan
        ax.imshow(output[chart], vmin=0, vmax=train_options['n_classes'][chart] - 2, cmap='jet', interpolation='nearest')
        ax.set_xticks([])
        ax.set_yticks([])
        chart_cbar(ax=ax, n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
    
    plt.suptitle(f"Scene: {scene_name}", y=0.65)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.5, hspace=-0)
    fig.savefig(f"inference/{scene_name}.png", format='png', dpi=128, bbox_inches="tight")
    plt.close('all')


# - Save upload_package with zlib compression.
print('Saving upload_package. Compressing data with zlib.')
compression = dict(zlib=True, complevel=1)
encoding = {var: compression for var in upload_package.data_vars}
upload_package.to_netcdf('upload_package.nc', mode='w', format='netcdf4', engine='netcdf4', encoding=encoding)
print('Testing completed.')

Testing.


  0%|          | 0/20 [00:00<?, ?it/s]

Saving upload_package. Compressing data with zlib.
Testing completed.
